In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

### 1. creating search algorithm for amazon

In [41]:
# starting chrome 
driver = webdriver.Chrome('chromedriver')
#driver.get("https://www.amazon.in/")
delay = 10
WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'nav_last')))

NameError: name 'EC' is not defined

In [83]:
driver.get('http://www.amazon.in/')

In [96]:
# navigating to search box
search_box = driver.find_element_by_xpath("//input[@id = 'twotabsearchtextbox']")
search_box.click()
search_box.clear()
search_box.send_keys("guitar")
search_btn = driver.find_element_by_xpath("//input[@id = 'nav-search-submit-button']")
search_btn.click()

### 2. scraping the data of guitars from amazon

In [97]:
links = []
for i in range(2):
    link = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']/div/h2/a") 
    link = [g.get_attribute("href") for g in link]
    links = links + link
    driver.find_element_by_xpath("//li[@class = 'a-last']/a").click()
    time.sleep(10)

In [99]:
len(links)

129

In [106]:
title = []
brand = []
rating = []
no_ratings = []
price = []
product_url = []
delivery_date = []
color = []
material = []
returns_policy = []

for i in range(5):
    driver.get(links[i])
    time.sleep(5)
    title.append(driver.find_element_by_xpath("//span[@id = 'productTitle']").text)
    brand.append(driver.find_element_by_xpath("//tr[@class= 'a-spacing-small']/td[2]/span").text)
    
    try:
        rating.append(driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']").text.split()[0])
    except NoSuchElementException:
        rating.append("-")
    
    try:
        no_ratings.append(driver.find_element_by_xpath("//div[@class='a-row a-spacing-medium averageStarRatingNumerical']/span").text.split()[0].replace(',',""))
    except NoSuchElementException:
        no_ratings.append("-")
        
    try:
        price.append(driver.find_element_by_xpath("//span[@id='priceblock_dealprice']").text.replace('₹',""))
    except NoSuchElementException:
        price.append("-")
    
    product_url.append(links[i])
    
    try:
        delivery_date.append(driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b").text)
    except NoSuchElementException:
        delivery_date.append("-")
        
    try:
        details = driver.find_elements_by_xpath("//tbody")[1].text
        color.append(details.split("\n")[1].split()[-1])
        material.append(details.split("\n")[2].split()[-1])
    except IndexError:
        color.append('-')
        material.append('-')
        
    try:
        returns_policy.append(driver.find_element_by_xpath("//div[@id = 'RETURNS_POLICY']/span/div[2]/a").text)
    except NoSuchElementException:
        returns_policy.append("-")
    
    #rating.append(driver.find_element_by_xpath("//i[@class='a-icon a-icon-star a-star-4']/span").text.split(" ")[1])

In [107]:
dict = {"brand":brand,"title":title,"rating":rating, "no of ratings":no_ratings, "price":price, "color":color, 'material':material, 'delivery_date':delivery_date, "returns_policy":returns_policy, "product_url":product_url}

In [108]:
df = pd.DataFrame(dict)

In [109]:
df.head()

,brand,title,rating,no of ratings,price,color,material,delivery_date,returns_policy,product_url
0,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4,13319,"1,999.00",Black,Wood,"Thursday, Oct 28",7 Days Replacement,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9,1042,"6,599.00",-,-,"Friday, Oct 29",7 Days Replacement,https://www.amazon.in/gp/slredirect/picassoRed...
2,JUAREZ,"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...",4,4484,"1,999.00",Sunburst,Wood,"Monday, Oct 25",7 Days Replacement,https://www.amazon.in/gp/slredirect/picassoRed...
3,JUAREZ,"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...",4,4349,"1,999.00",Sunburst,Wood,"Thursday, Oct 28",7 Days Replacement,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4,13319,"1,999.00",Black,Wood,"Thursday, Oct 28",7 Days Replacement,https://www.amazon.in/Juarez-Acoustic-Cutaway-...


### 3. Accessing the search bar on google image

In [57]:
driver = webdriver.Chrome('chromedriver')

#### 1. fruits

In [82]:
# finding the search box and searching the key words
driver.get('https://images.google.com/')
search_box = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_box.click()
search_box.clear()
search_box.send_keys("fruits")
search_box.submit()
time.sleep(3)
for i in range(2):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") #scrolling down the page
    time.sleep(3)

In [83]:
fruit_img = driver.find_elements_by_xpath("//div[@class = 'bRMDJf islir']/img")
len(fruit_img)

200

In [85]:
fruit_img_url = [image.get_attribute("src") for image in fruit_img][:100]
fruit_df= pd.DataFrame({"fruit image url":fruit_img_url})
fruit_df.head()

,fruit image url
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


#### 2. cars

In [73]:
driver.get('https://images.google.com/')
time.sleep(3)
search_box = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_box.click()
search_box.clear()
search_box.send_keys("cars")
search_box.submit()
time.sleep(3)
for i in range(2):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") #scrolling down the page
    time.sleep(3)

In [75]:
car_img = driver.find_elements_by_xpath("//div[@class = 'bRMDJf islir']/img")
len(car_img)

200

In [78]:
car_img_url = [image.get_attribute("src") for image in img][:100]
car_df= pd.DataFrame({"car image url":car_img_url})

In [81]:
car_df.head()

,car image url
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


#### 3. machine learning

In [86]:
driver.get('https://images.google.com/')
time.sleep(3)
search_box = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_box.click()
search_box.clear()
search_box.send_keys("Machine learning")
search_box.submit()
time.sleep(3)
for i in range(2):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") #scrolling down the page
    time.sleep(3)

In [88]:
ML_img = driver.find_elements_by_xpath("//div[@class = 'bRMDJf islir']/img")
ML_img_url = [image.get_attribute("src") for image in ML_img][:100]
ML_df= pd.DataFrame({"ML image url":ML_img_url})
ML_df.head()

,ML image url
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


### 4. fetching the details of a smartphone from flipkart

In [2]:
driver = webdriver.Chrome('chromedriver')
driver.get('http://www.flipkart.com/')

In [3]:
search_box = driver.find_element_by_xpath("//input[@class = '_3704LK']")
search_box.send_keys("Redmi note 10s")
search_box.submit()
time.sleep(3)

In [4]:
phones = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
phone_links = [phone.get_attribute('href') for phone in phones] # collecting links

In [14]:
brand = []
name = []
color = []
RAM = []
storage = []
camera1 = []
camera2 = []
dis_size = []
dis_res = []
pros = []
pros_c = []
battery = []
price = []
url = []

for i in range(len(phone_links)):
#for i in range(4):
    driver.get(phone_links[i])
    time.sleep(3)
    url.append(phone_links[i])                                                          # adding url
    brand.append("REDMI")
    title = driver.find_element_by_xpath("//h1[@class = 'yhB1nd']/span")
    name.append(" ".join((title.text.split("(")[0].split())[1:]))                       # name
    color.append(title.text.split("(")[1].split(",")[0])                                # color
    price.append(driver.find_element_by_xpath("//div[@class = '_30jeq3 _16Jk6d']").text)     # price
    detail = driver.find_element_by_xpath("//div[@class = '_2418kt']")                  
    RAM.append(" ".join(detail.text.split("\n")[0].split("|")[0].split()[:2]))          # RAM
    storage.append(" ".join(detail.text.split("\n")[0].split("|")[1].split()[:2]))      # storage
    dis_size.append(detail.text.split("\n")[1].split("(")[0].strip())                   # display size
    battery.append(detail.text.split("\n")[3].replace("Battery","").strip())            # battery capacity
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()          # clicking on "Read more"
    time.sleep(3)
    dis_res.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody").text.split("\n")[3])    # display resolution
    OS = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody").text.split('\n') 
    keys = []
    values = []
    for i in range(len(OS)):
        if i%2==0:
            keys.append(OS[i])
        else:
            values.append(OS[i])
    OSs = dict(zip(keys, values))
    try:
        pros.append(OSs["Processor Type"])                                              # processor
    except:
        pros.append("-")
    pros_c.append(OSs["Processor Core"])                                                # cores
    CAM = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody").text.split('\n')
    keys2 = []
    values2  = []
    for i in range(len(CAM)):
        if i%2==0:
            keys2.append(CAM[i])
        else:
            values2.append(CAM[i])
    CAMs = dict(zip(keys2, values2))
    camera1.append(CAMs['Primary Camera'])                                              # primary camera
    try:
        camera2.append(CAMs['Secondary Camera'])                                        # secondary camera
    except:
        camera2.append("-")

In [15]:
dictionary = {"brand":brand, "color":color,"name":name, "RAM":RAM, "Storage":storage, "dis_size":dis_size, "display resolution": dis_res,"battery":battery,
             "processor": pros, "cores":pros_c,"primary camera": camera1, "secondary camera": camera2, "price":price, "url":url}

In [16]:
df = pd.DataFrame(dictionary)
df.head()

,brand,color,name,RAM,Storage,dis_size,display resolution,battery,processor,cores,primary camera,secondary camera,price,url
0,REDMI,Cosmic Purple,Note 10S,6 GB,64 GB,16.33 cm,2400 x 1080$$pixels,5000 mAh,MediaTek Helio G95,Octa Core,64MP + 8MP + 2MP + 2MP,13MP Front Camera,"₹14,793",https://www.flipkart.com/redmi-note-10s-cosmic...
1,REDMI,Cosmic Purple,Note 10S,6 GB,128 GB,16.33 cm,2400 x 1080$$pixel,5000 mAh,MediaTek Helio G95,Octa Core,64MP + 8MP + 2MP + 2MP,13MP Front Camera,"₹16,743",https://www.flipkart.com/redmi-note-10s-cosmic...
2,REDMI,Deep Sea Blue,Note 10S,6 GB,128 GB,16.33 cm,2400 x 1080 Pixels,5000 mAh,MediaTek Helio G95,Octa Core,64MP + 8MP + 2MP + 2MP,13MP Front Camera,"₹16,438",https://www.flipkart.com/redmi-note-10s-deep-s...
3,REDMI,Frost White,Note 10S,6 GB,128 GB,16.33 cm,2400 x 1080 Pixels,5000 mAh,MediaTek Helio G95,Octa Core,64MP + 8MP + 2MP + 2MP,13MP Front Camera,"₹16,345",https://www.flipkart.com/redmi-note-10s-frost-...
4,REDMI,Frost White,Note 10S,6 GB,64 GB,16.33 cm,2400 x 1080 Pixels,5000 mAh,MediaTek Helio G95,Octa Core,64MP + 8MP + 2MP + 2MP,13MP Front Camera,"₹14,787",https://www.flipkart.com/redmi-note-10s-frost-...


In [24]:
# saving as CSV
df.to_csv("smartphone.csv")

### 5. fetcing the coordinates of a city from google map

In [17]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.google.co.in/maps/")

In [22]:
cities = ["chandigarh", "delhi", "bangalore", "abohar", "shimla", "kanpur", "mumbai"] # list of some cities
coordinates = []
for city in cities:
    map_search = driver.find_element_by_xpath("//div[@id='gs_lc50']/input[1]")
    map_search.clear()
    map_search.send_keys(city)
    driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']").click()
    time.sleep(10)
    current_url = driver.current_url                                            # fetching the curent url
    coordinates.append(current_url.split("@")[1].split("/")[0].split(",")[:2])  # extrarcting the coordinates from url

In [26]:
map_dictionary = dict(zip(cities, coordinates))
city_map_df = pd.DataFrame(map_dictionary).T
city_map_df.columns = ["latitude", "longitude"]
city_map_df.head()

,latitude,longitude
chandigarh,30.7352102,76.693488
delhi,28.6472799,76.8130611
bangalore,28.6472799,76.8130611
abohar,28.6472799,76.8130611
shimla,28.6472799,76.8130611


### 6. fetching the funding deal data

In [8]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://trak.in/india-startup-funding-investment-2015/")
time.sleep(10)
for i in range(3):
    x = i+48
    name = "tablepress-"+str(x)+"_length"
    select = Select(driver.find_element_by_name(name))                        # selecting maximum from dropdown
    select.select_by_value('100')
t1 = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td")# fetching the table wise data
t2 = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td")
t3 = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td")

raw_data = t1+t2+t3                                                           # concatenating the raw data
table = [r.text for r in raw_data]                                            # fetching the text data

In [9]:
date = []
startup_name = []
industry = []
sub_vertical = []
location = []
investors = []
investment_type = []
amount = []

# diversifying the series of data into their respective column
i = 0
while i < len(table):
    date.append(table[i+1])
    startup_name.append(table[i+2])
    industry.append(table[i+3])
    sub_vertical.append(table[i+4])
    location.append(table[i+5])
    investors.append(table[i+6])
    investment_type.append(table[i+7])
    amount.append(table[i+8])
    i = i+9

In [10]:
fund_dict = {"date":date, "startup name": startup_name, "industry":industry, "sub vertical":sub_vertical, 
                "location":location, "investors":investors, "investment_type":investment_type, "amount(USD)":amount}

fund_df = pd.DataFrame(fund_dict)
fund_df.head()

,date,startup name,industry,sub vertical,location,investors,investment_type,amount(USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"


In [13]:
fund_df.shape

(33, 8)

### 7. fetching the best gaming laptop from digit.in

In [7]:
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")
time.sleep(10)

In [8]:
title = driver.find_elements_by_xpath("//h3")
table = driver.find_elements_by_xpath("//table[@id='summtable']/tbody/tr/td[@class='smprice']")
price = [lap.text for lap in table]

In [10]:
specs = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr")
specs = [spec.text for spec in specs if len(spec.text)>0]
specs = [item.split(":") for item in specs if item !="SPECIFICATION"]
specs = [spec for spec in specs if spec[0]!="Price "]

In [11]:
OS = []
display = []
processor = []
memory = []
weight = []
dimension = []
graphic_processor = []

i = 0
while i < len(specs):
    OS.append(specs[i][1])
    display.append(specs[i+1][1])
    processor.append(specs[i+2][1])
    memory.append(specs[i+3][1])
    weight.append(specs[i+4][1])
    dimension.append(specs[i+5][1])
    graphic_processor.append(specs[i+6][1])
    i=i+7
    
lap_dict = {"Operating system":OS, "display":display, "processor":processor,"memory":memory, "weight":weight, "dimension":dimension
           ,"graphic_processor":graphic_processor, "price":price}

lap_df = pd.DataFrame(lap_dict)
lap_df.head()

,Operating system,display,processor,memory,weight,dimension,graphic_processor,price
0,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"₹ 129,990"
1,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"₹ 134,990"
2,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"₹ 268,990"
3,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"₹ 342,989"
4,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"₹ 319,990"


### 8. fetching billionaires from forbes

In [12]:
bil = 'https://www.forbes.com/billionaires/'
driver = webdriver.Chrome('chromedriver')
driver.get(bil)
time.sleep(10)

In [18]:
table = driver.find_elements_by_xpath("//div[@class='table-row-group']/div[@class='table-row ']")
text = [t.text.split('\n') for t in table]

In [21]:
rank = []
name = []
net_worth = []
age = []
citizenship = []
source = []
industry = []

for item in text:
    rank.append(item[0])
    name.append(item[1])
    net_worth.append(item[2])
    age.append(item[3])
    citizenship.append(item[4])
    source.append(item[5])
    industry.append(item[6])
    
bil_dict = {"rank":rank, "name":name, "net worth":net_worth, "age":age, "citizenship":citizenship, "source":source, "industry":industry}
bil_df = pd.DataFrame(bil_dict)
bil_df.head()

,rank,name,net worth,age,citizenship,source,industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology


### 9. fetching comments from youtube video 

In [66]:
youtube = 'https://www.youtube.com/watch?v=lotbZQ55SgU'
driver = webdriver.Chrome('chromedriver')
driver.get(youtube)
time.sleep(10)

In [67]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
content = driver.find_elements_by_id("content-text")

In [79]:
while len(comment) < 501:
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
    content = driver.find_elements_by_id("content-text")
    comment = [c.text for c in content]
    time_ago = driver.find_elements_by_xpath("//div[@id='header-author']/yt-formatted-string")
    t = [t.text for t in time_ago]

In [72]:
len(comment)

520

In [74]:
votes = driver.find_elements_by_xpath("//span[@id='vote-count-middle']")
votes = [t.text for t in votes]

for i in range(len(votes)):
    if len(votes[i])==0:
        votes[i]='-'

In [77]:
n = min(len(comment), len(time_ago), len(votes))
comment_dict = {"comment":comment[:n], "time_ago":t[:n], "votes":votes[:n]}

In [78]:
comment_df = pd.DataFrame(comment_dict)
comment_df.head()

,comment,time_ago,votes
0,Well the pyramids were built before Isaac Newt...,17K,17K
1,I love how a single youtuber can make all this...,1.2K,1.2K
2,"""Work so hard that people in the future believ...",1.5K,1.5K
3,Imagine how beautiful the pyramids looked like...,267,267
4,I love that even in this age we still can't fi...,516,516


### 10. fetching hostels data

In [ ]:
hostel = "https://www.hostelworld.com/"
driver  = webdriver.Chrome('chromedriver')
driver.get(hostel)
time.sleep(5)

In [ ]:
search_host = driver.find_element_by_xpath("//input[@placeholder = 'Where do you want to go?']")
search_host.clear()
search_host.send_keys("london")

In [ ]:
driver.find_element_by_xpath("//li[@tabindex = '-1'][2]").click()
time.sleep(3)
driver.find_element_by_xpath("//button[@id='search-button']").click()
time.sleep(3)

In [ ]:
title = driver.find_elements_by_xpath("//div[@class='property']/div/h2/a")
title = [t.text for t in title]
distance = driver.find_elements_by_xpath("//div[@class='subtitle body-3']/a/span[1]")
distance = [d.text.split("-")[1].split()[0] for d in distance]
